<a href="https://colab.research.google.com/github/daniivelascoo/ifp-programacion-ia/blob/main/Teoria_1_3_C_Agrupacion_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Teoría Bloque C: El Arte de Agrupar (GroupBy)
**Sprint 1.3 - Pandas Deep Dive**

Hasta ahora hemos mirado los datos fila a fila. Pero las preguntas de negocio suelen ser sobre **grupos**:
*   *¿Qué tienda vende más?*
*   *¿Cuál es el salario medio por departamento?*

Para esto usamos la técnica **Split-Apply-Combine** (Dividir-Aplicar-Combinar).

### 🎯 Objetivos:
1.  **GroupBy Simple:** Agrupar por una categoría y calcular medias o sumas.
2.  **Agg (Agregación):** Calcular varias estadísticas a la vez (Mínimo, Máximo y Media).
3.  **Pivot Table:** Crear tablas dinámicas cruzando dos variables.

## 1. Preparación de Datos

Vamos a crear un dataset de empleados con Departamentos, Ventas y Antigüedad.

In [1]:
import pandas as pd
import numpy as np

# Creamos un DataFrame
df = pd.DataFrame({
    'Depto': ['Ventas', 'Ventas', 'IT', 'IT', 'RRHH', 'Ventas'],
    'Empleado': ['Ana', 'Carlos', 'Eva', 'Juan', 'Luisa', 'Pedro'],
    'Ciudad': ['Madrid', 'Madrid', 'Barcelona', 'Madrid', 'Valencia', 'Barcelona'],
    'Ventas': [20000, 22000, 0, 0, 0, 25000],  # IT y RRHH no venden
    'Salario': [1500, 1600, 3000, 3200, 1800, 1700]
})

print("--- Tabla de Empleados ---")
display(df)

--- Tabla de Empleados ---


,Depto,Empleado,Ciudad,Ventas,Salario
0,Ventas,Ana,Madrid,20000,1500
1,Ventas,Carlos,Madrid,22000,1600
2,IT,Eva,Barcelona,0,3000
3,IT,Juan,Madrid,0,3200
4,RRHH,Luisa,Valencia,0,1800
5,Ventas,Pedro,Barcelona,25000,1700


## 2. Agrupación Simple (GroupBy)

Queremos saber: **¿Cuál es el salario medio de cada departamento?**
1.  Agrupamos por `Depto`.
2.  Seleccionamos la columna `Salario`.
3.  Aplicamos la función `mean()` (Media).

In [2]:
# Sintaxis: df.groupby('COLUMNA_GRUPO')['COLUMNA_VALOR'].funcion()

# 1. Agrupar
# Calcula la media de Salario por Depto
salario_medio = df.groupby('Depto')['Salario'].mean() # TODO: Pon la columna 'Depto'

print("\n--- Salario Medio por Departamento ---")
display(salario_medio)

# 2. Ordenar
# ¿Qué departamento cobra más? Ordenamos descendente.
ranking = salario_medio.sort_values(ascending=False) # TODO: Pon False para descendente
display(ranking)


--- Salario Medio por Departamento ---


,Salario
Depto,
IT,3100.0
RRHH,1800.0
Ventas,1600.0


,Salario
Depto,
IT,3100.0
RRHH,1800.0
Ventas,1600.0


## 3. Agregación Avanzada (.agg)

A veces la media no es suficiente. Queremos un informe completo:
*   ¿Cuánto gastamos en total en sueldos? (`sum`)
*   ¿Cuál es el sueldo máximo? (`max`)
*   ¿Cuántos empleados hay? (`count`)

Para hacer todo a la vez, usamos `.agg()`.

In [3]:
# Queremos un resumen del Salario por Departamento
# Pasamos una lista de funciones: ['min', 'max', 'mean', 'sum', 'count']

reporte = df.groupby('Depto')['Salario'].agg(['min', 'max', 'mean']) # TODO: Añade 'mean'

print("--- Informe Salarial Completo ---")
display(reporte)

# TRUCO: Agregación por diccionario (Distintas funciones para distintas columnas)
# - Ventas: Queremos la SUMA total.
# - Empleado: Queremos la CUENTA (cuántos hay).

reporte_mix = df.groupby('Depto').agg({
    'Ventas': 'sum',   # TODO: Función 'sum'
    'Empleado': 'count'  # TODO: Función 'count'
})

print("\n--- Informe Mixto ---")
display(reporte_mix)

--- Informe Salarial Completo ---


,min,max,mean
Depto,,,
IT,3000,3200,3100.0
RRHH,1800,1800,1800.0
Ventas,1500,1700,1600.0



--- Informe Mixto ---


,Ventas,Empleado
Depto,,
IT,0,2
RRHH,0,1
Ventas,67000,3


## 4. Tablas Dinámicas (Pivot Table)

¿Y si queremos cruzar **dos variables**?
Ejemplo: Salario medio desglosado por **Departamento** (Filas) y **Ciudad** (Columnas).
Esto en Excel es una Tabla Dinámica. En Pandas es `pivot_table`.

In [4]:
# Sintaxis:
# index = Lo que va en las filas
# columns = Lo que va en las columnas
# values = Los números que queremos calcular
# aggfunc = La operación (media, suma...)

tabla_dinamica = df.pivot_table(
    index='Depto',      # TODO: Filas = 'Depto'
    columns='Ciudad',    # TODO: Columnas = 'Ciudad'
    values='Salario',     # Qué calculamos
    aggfunc='mean'        # La media
)

print("--- Tabla Dinámica (Salarios) ---")
display(tabla_dinamica)

# Fíjate que salen algunos NaN.
# (Ej: No hay nadie de RRHH en Barcelona, por eso sale NaN).
# Podemos rellenarlo con fill_value=0 dentro del pivot_table.

--- Tabla Dinámica (Salarios) ---


Ciudad,Barcelona,Madrid,Valencia
Depto,,,
IT,3000.0,3200.0,NaN
RRHH,NaN,NaN,1800.0
Ventas,1700.0,1550.0,NaN
